In [51]:
# sales by year
def sales_by_year(df_clean):
    df_copy = df_clean.copy()
    df_copy["year"] = df_copy["order_date"].dt.year
    return df_copy.groupby("year")["sales"].sum()

# profit margin by year
def profit_margin_by_year(df_clean):
    df_copy = df_clean.copy()
    df_copy["year"] = df_copy["order_date"].dt.year
    yearly = df_copy.groupby("year")[["sales", "profit"]].sum()
    yearly["profit_margin"] = yearly["profit"] / yearly["sales"]
    return yearly["profit_margin"]

# sales by month
def sales_by_month(df_clean):
    df_clean = df_clean.copy()
    df_clean['month'] = df_clean['order_date'].dt.to_period('M')
    monthly = df_clean.groupby('month')['profit'].sum()
    return monthly

# top cities
def top_cities(df, top_n=10):
    df_copy = df.copy()
    city_data = df_copy.groupby('city')[['sales', 'profit']].sum()
    return city_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider, Dropdown
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/ayushinto-space/Performance-Metrics-and-Sales-Analysis-Using-Python/main/Sales_Cleaned.csv")  # Load your data          # Run the interactive dashboard
df['order_date'] = pd.to_datetime(df['order_date'])

def plot_top_cities(df, top_n=10, metric='sales'):
    city_data = top_cities(df, top_n=top_n)
    city_data = city_data.sort_values(metric, ascending=False).head(top_n)
    plt.figure(figsize=(10,5))
    sns.barplot(x=city_data.index, y=city_data[metric], palette='coolwarm')
    plt.xticks(rotation=45, ha='right')
    plt.title(f"Top {top_n} Cities by {metric.capitalize()}")
    plt.xlabel("City")
    plt.ylabel(metric.capitalize())
    plt.tight_layout()
    plt.show()


def plot_sales_by_year(df_clean):
    yearly_sales = sales_by_year(df_clean)
    plt.figure(figsize=(7, 4.5))
    sns.barplot(x=yearly_sales.index, y=yearly_sales.values, palette="PuBuGn", hue=None, legend=False)
    plt.title("Total Sales by Year", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Sales", fontsize=12)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    sns.despine()
    plt.tight_layout()
    plt.show()

def plot_profit_margin_by_year(df_clean):
    yearly_margin = profit_margin_by_year(df_clean)
    plt.figure(figsize=(7, 4.5))
    sns.lineplot(x=yearly_margin.index, y=yearly_margin.values*100, marker='o', linewidth=2,
                 markersize=7, color="crimson")
    plt.title("Profit Margin by Year", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Profit Margin", fontsize=12)
    plt.xticks([2023, 2023.5, 2024, 2024.5, 2025])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    sns.despine()
    plt.tight_layout()
    plt.show()

def plot_sales_by_month(df_clean):
    monthly = sales_by_month(df_clean)
    plt.figure(figsize=(7,4))
    plt.plot(monthly.index.astype(str), monthly.values)
    plt.xticks(rotation=45)
    plt.title("Monthly Sales Trend")
    plt.xlabel("Month")
    plt.ylabel("Total Sales")
    plt.tight_layout()
    plt.show()

def interactive_sales_dashboard(df_clean):
    plot_options = {
        "Sales by Year": plot_sales_by_year,
        "Profit Margin by Year": plot_profit_margin_by_year,
        "Sales by Month": plot_sales_by_month,
        "Top Cities by Sales": None  # special case
    }

    @interact(plot_type=Dropdown(options=list(plot_options.keys()), description="Select Plot"))
    def update(plot_type):
        if plot_type == "Top Cities by Sales":
            # call the original interactive Top Cities dashboard
            @interact(top_n=IntSlider(min=3, max=20, step=1, value=10, description="Top N Cities"),
                      metric=Dropdown(options=["sales","profit"], value="sales", description="Metric"))
            def top_cities_interactive(top_n, metric):
                plot_top_cities(df_clean, top_n=top_n, metric=metric)
        else:
          plot_func = plot_options[plot_type]
          plot_func(df_clean)


interactive_sales_dashboard(df)

